# Карьерный блок
# Задача
Вы работаете в агентстве недвижимости «Х». Для того чтобы получить наибольшую комиссию в наименьший срок, необходимо четко понимать справедливую рыночную стоимость объекта недвижимости или стоимость его аренды. 
К вам обратился руководитель компании с просьбой 

# Разработать модель предсказания справедливой рыночной цены аренды квартиры по заданным параметрам. 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import datetime
import calendar
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [19]:
df=pd.read_excel(r'C:\csv\dz.xlsx')
display(df)

,id,create_date,rent_price,deposit,floor,area_total,rooms,region,repair_type,agent_fee
0,1,2021-12-06 09:54:22.001,88400.0,NaN,2,63.2,2,1,NaN,NaN
1,2,2021-12-06 09:52:18.815,88100.0,NaN,6,53.6,2,1,NaN,NaN
2,3,2021-12-06 09:48:26.329,36000.0,NaN,9,39.0,1,1,NaN,NaN
3,4,2021-12-06 09:46:42.017,35000.0,NaN,5,40.0,1,1,NaN,NaN
4,5,2021-12-06 09:45:50.589,30000.0,NaN,8,28.0,0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4995,4996,2021-12-01 21:06:05.572,33000.0,33000.0,5,37.0,1,1,cosmetic,60.0
4996,4997,2021-12-01 21:04:25.275,35000.0,35000.0,1,46.0,2,2,cosmetic,0.0
4997,4998,2021-12-01 21:04:11.731,40000.0,40000.0,2,54.3,2,2,unknown,0.0
4998,4999,2021-12-01 21:03:25.016,30000.0,30000.0,10,61.0,2,2,cosmetic,0.0


# 1 Провести базовую предварительную обработку данных: заполнить пропуски, очистить данные от дубликатов, провести масштабирование.

# Ищем пропуски

In [11]:
df.isnull().sum() 

id              0
create_date     0
rent_price      0
deposit        88
floor           0
area_total      0
rooms           0
region          0
repair_type    88
agent_fee      88
dtype: int64

# Что есть что

In [12]:
df.repair_type.unique()

array([nan, 'design', 'cosmetic', 'euro', 'unknown', 'no'], dtype=object)

In [13]:
df.agent_fee.unique()

array([ nan,  40.,  50.,  70.,   0., 100.,  99.,  30.,  60.,  45.,  80.,
        25.,  90.,   1.,  35.,  10.,  72.,  15.,  20.,  95.,  65.,  97.,
        69.,  55.,  47.,  61.,  54.,  29.,  84.,  62.,  26.,  75.,  85.,
        67.,  12.,  32.,  53.,  28.,  38.,  37.,  88.,  17.])

In [14]:
df.describe()

,id,create_date,rent_price,deposit,floor,rooms,region,agent_fee
count,5000.000000,5000,5000.000000,4.912000e+03,5000.000000,5000.000000,5000.000000,4912.000000
mean,2500.500000,2021-12-03 22:20:59.184492032,56969.306000,7.247456e+04,7.405400,1.615600,1.250200,42.043770
min,1.000000,2021-12-01 21:02:57.189000,6300.000000,0.000000e+00,1.000000,-1.000000,1.000000,0.000000
25%,1250.750000,2021-12-02 18:34:32.942749952,35000.000000,3.000000e+04,3.000000,1.000000,1.000000,0.000000
50%,2500.500000,2021-12-03 18:27:10.567500032,45000.000000,4.300000e+04,6.000000,2.000000,1.000000,50.000000
75%,3750.250000,2021-12-04 21:12:04.206749952,60000.000000,6.000000e+04,10.000000,2.000000,2.000000,50.000000
max,5000.000000,2021-12-06 09:54:22.001000,300000.000000,1.000000e+08,55.000000,6.000000,2.000000,100.000000
std,1443.520003,NaN,41921.368672,1.426763e+06,5.627769,0.885208,0.433171,32.951788


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           5000 non-null   int64         
 1   create_date  5000 non-null   datetime64[ns]
 2   rent_price   5000 non-null   float64       
 3   deposit      4912 non-null   float64       
 4   floor        5000 non-null   int64         
 5   area_total   5000 non-null   object        
 6   rooms        5000 non-null   int64         
 7   region       5000 non-null   int64         
 8   repair_type  4912 non-null   object        
 9   agent_fee    4912 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(2)
memory usage: 390.8+ KB


# Действия с пропусками и выбросами

1. rooms       - 0 и -1     - заменим на 1
2. deposit     - пропуски   - заменим на 0
3. area_total  - месяца     - заменим на цифру, соотв-ю номеру месяца, переведем в float
4. repair_type - пропуски   - заменим на unknown
5. agent_fee   - пропуски   - заменим на 0